# Data Preprocessing

This notebook is intended to process data in the "downloaded" folder into tensorized form, particularly aqs and meteorology data.

## 1. Join AQ and Met datasets

In [49]:
import pandas as pd
import json
import os
import numpy as np

PARAM_DICT = {
    "44201": "o3",
    "68105": "avg_temp",
    "62101": "outdoor_temp",
    "61301": "mix_height",
    "42101": "co",
    "42601": "no",
    "42602": "no2",
    "88101": "pm25",
    "86101": "pm10_25"
}

In [50]:
df_list:list[pd.DataFrame] = []

for dataset in os.listdir("./data/downloaded/aqs/"):
    if "json" not in dataset: continue
    with open(os.path.join("./data/downloaded/aqs/", dataset)) as f:
        raw_json = json.load(f)
    df = pd.json_normalize(raw_json['Data'])
    df['datetime_gmt'] = pd.to_datetime(df['date_gmt'] + ' ' + df['time_gmt'], utc=True)

    df = df[['datetime_gmt','parameter_code','sample_measurement']]
    df['parameter_code'] = df['parameter_code'].apply(lambda x: PARAM_DICT[x])
    df = df.pivot_table(
        index='datetime_gmt',
        columns='parameter_code',
        values='sample_measurement',
        dropna=False
    ).reset_index()
    df = df.drop(columns=['avg_temp','mix_height', 'pm10_25'], errors='ignore')
    df_list.extend([df])

aq_ds_primary = pd.concat(df_list)
aq_ds_primary['datetime_gmt'] = pd.to_datetime(aq_ds_primary['datetime_gmt'], utc=True, errors='raise')
aq_ds_primary = aq_ds_primary.set_index('datetime_gmt')
aq_ds_primary.to_pickle('./data/downloaded/myron_ds.pkl')
aq_ds_primary.to_json('./data/downloaded/myron_ds.json', index=True)

In [72]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

worldmet = importr("worldmet")

ro.r('''
	# met_ds = worldmet::importNOAA(code=c("997278-99999","725070-14765"), year=2014:2025)
	met_ds = worldmet::importNOAA(code=c("725070-14765"), year=2014:2025)
''')

met_ds_primary:pd.DataFrame
with (ro.default_converter + pandas2ri.converter).context():
    met_ds_primary = ro.r['met_ds'] # type: ignore

 ■■■                                8% |  ETA: 29s
 ■■■■■■                            17% |  ETA: 26s
 ■■■■■■■■■                         25% |  ETA: 24s
 ■■■■■■■■■■■                       33% |  ETA: 21s
 ■■■■■■■■■■■■■                     42% |  ETA: 19s
 ■■■■■■■■■■■■■■■■                  50% |  ETA: 16s
 ■■■■■■■■■■■■■■■■■■■               58% |  ETA: 13s
 ■■■■■■■■■■■■■■■■■■■■■             67% |  ETA: 11s
 ■■■■■■■■■■■■■■■■■■■■■■■           75% |  ETA:  8s
 ■■■■■■■■■■■■■■■■■■■■■■■■■■        83% |  ETA:  5s
 ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■     92% |  ETA:  3s


In [82]:
met_ds = met_ds_primary
aq_ds  = aq_ds_primary
met_ds = met_ds.rename(columns={"date":"datetime"})
met_ds = met_ds.reset_index()
aq_ds  = aq_ds.reset_index()
aq_ds  = aq_ds .rename(columns={"datetime_gmt":"datetime"})

met_ds = met_ds.drop(
    columns=[
        'cl_2','cl_3','cl_2_height','cl_3_height','precip_6','pwc',
		'code','station','latitude','longitude','elev'
	]
)
met_ds = met_ds.rename(
    columns={
        "Uu":"ws_east_west",
        "Vv":"ws_north_south"
	}
)

met_ds['datetime'] = pd.to_datetime(met_ds['datetime'], utc=True)
aq_ds['datetime']  = pd.to_datetime(aq_ds['datetime'], utc=True)
ds = pd.merge_ordered(
    met_ds,
    aq_ds,
    on='datetime'
)

ds.to_json("data/joined_met_aq_ds.json", indent=2, index=True)
ds.to_pickle("data/joined_met_aq_ds.pckl")

## 2. Tensorize joined dataset

Convert dataset from pd.DataFrame to tf.Tensor; split and encode datetime information using differentiable function; process missing data.